In [ ]:
import numpy as np
import pandas as pd

import gensim
import string
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = nltk.corpus.stopwords.words('russian')
def preprocess(doc):
    # к нижнему регистру
    doc = doc.lower()
    # убираем пунктуацию, пробелы, прочее
    for p in string.punctuation + string.whitespace + 'http': 
        doc = doc.replace(p, ' ')
    # убираем кавычки
    for p in ['«', '»', '\'', '\"']:
        doc = doc.replace(p, ' ')
    for p in stop_words:
        doc = doc.replace(' ' + p + ' ', ' ')
    # убираем лишние пробелы, объединяем обратно
    words = doc.split()


   
    
    return words

In [ ]:
df = pd.read_csv('/content/news.csv')
df = df[df['rubric'].notna()]

In [ ]:
df = df.drop(['source','title','publication_date','subrubric','tags'],axis =1 )

In [ ]:
df

,text,rubric
0,В 1930-е годы Советский Союз охватила лихорадк...,Экономика
1,Олимпийская чемпионка по фигурному катанию Ал...,Спорт
2,Российский врач-диетолог Римма Мойсенко объясн...,Из жизни
3,В 2019 году телеканал «Ю» запустил адаптацию з...,Интернет и СМИ
4,Актер Михаил Ефремов систематически употребл...,Культура
...,...,...
4505,C 1 января Болгария начала получать российский...,Экономика
4506,В России вступили в силу новые правила содержа...,Россия
4507,Российский певец Сергей Лазарев объяснил сво...,Культура
4508,Президент США Дональд Трамп заявил о нежелан...,Мир


In [ ]:
df['rubric'].value_counts()

Россия               908
Мир                  718
Спорт                494
Экономика            420
Бывший СССР          394
Силовые структуры    220
Интернет и СМИ       211
Наука и техника      208
Культура             206
Из жизни             197
Путешествия          183
Ценности             141
Дом                  123
Нацпроекты            53
69-я параллель        27
Name: rubric, dtype: int64

Выбрал 2 рубрики для бинарной классификации 


In [ ]:
df['text'] = df['text'].map(preprocess)

docs = list(df['text'].values)

In [ ]:
w2v = gensim.models.Word2Vec(sentences = docs, # список из списков, в каждом str
                             size = 300, # размер пространства признаков
                             window = 5, # длина двигающегося окна
                             min_count=5, # минимальное кол-во слов в окне
                              
                             sg=1, # используем skip-gramm
                             iter=50) # кол-во итераций

In [ ]:
df = df.loc[(df['rubric'] == 'Мир') | (df['rubric'] == 'Россия')]
#Оставил только 2 рубрики для бинарной классификации

In [ ]:
vector = w2v.wv.get_vector('россия')

In [ ]:
vector

array([-1.61289588e-01, -5.45829535e-01, -2.20198095e-01,  3.63287300e-01,
       -1.30189583e-01,  7.44966149e-01,  1.84165448e-01,  5.72970882e-02,
        7.79301375e-02,  3.04318428e-01, -5.69819689e-01,  5.54751456e-01,
       -1.87646344e-01,  2.80744761e-01, -5.53612530e-01,  9.57258940e-02,
       -3.11739207e-03,  1.64042979e-01,  2.22425815e-02, -4.18082863e-01,
       -4.47781891e-01,  3.10273975e-01, -4.49311823e-01,  1.35721520e-01,
        1.87423483e-01,  2.11014867e-01,  1.06136963e-01, -4.37411636e-01,
       -2.96020031e-01, -1.54076114e-01,  6.95673704e-01, -2.36655891e-01,
        4.27475244e-01,  3.73083681e-01, -4.82880265e-01,  7.79307067e-01,
       -5.65028906e-01,  2.03841731e-01,  1.24685131e-01,  2.29135320e-01,
        2.14554116e-01, -7.18565956e-02,  6.64239585e-01, -3.53487849e-01,
        1.04368618e-02,  5.19015431e-01, -6.53775513e-01, -3.12402844e-01,
        1.61327705e-01, -1.90802440e-01,  1.18759302e-02, -1.97168346e-02,
       -7.41018057e-01,  

In [ ]:
def mean_vector(string):
  len = 0
  vector = np.zeros(300)
  for word in string:
    if type(word) != str:
      continue
    elif word not in w2v.wv.vocab:
      continue
    vector = vector + w2v.wv.get_vector(word)
    len+=1


  return vector/len

In [ ]:
df['text'] = df['text'].map(mean_vector)

<ipython-input-28-b294a5bc2ff6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].map(mean_vector)


In [ ]:
vector_df = pd.DataFrame(df['text'].to_list(), columns=[f'feature_{i+1}' for i in range(300)])

# Concatenate feature DataFrame with rubric column
result  = pd.concat([df.reset_index().drop("index", axis=1), vector_df], axis=1)

In [ ]:
result

,text,rubric,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300
0,"[-0.1419259850351539, 0.08997282496064293, -0....",Мир,-0.141926,0.089973,-0.182864,0.091670,0.175368,0.184784,0.028104,-0.156278,...,-0.007197,-0.045465,0.029268,0.089884,-0.115882,-0.180595,-0.045483,-0.030673,0.007729,-0.050785
1,"[-0.17873677493087828, 0.03361459958204438, -0...",Россия,-0.178737,0.033615,-0.005492,-0.014667,0.077261,0.235475,-0.149112,-0.149998,...,-0.062148,-0.155750,0.076475,-0.038653,-0.199117,-0.201556,-0.074330,0.007207,-0.005383,-0.072535
2,"[-0.03762282231236221, 0.12408353854143234, -0...",Мир,-0.037623,0.124084,-0.031329,0.059092,0.144818,0.182589,-0.097897,-0.219109,...,-0.075188,-0.130190,0.100777,-0.012934,-0.165758,-0.021185,-0.081217,-0.030957,0.069550,-0.087013
3,"[-0.07561444902149517, 0.032000629464164376, -...",Мир,-0.075614,0.032001,-0.051134,0.102946,0.143312,0.287640,-0.108037,-0.233706,...,-0.035773,-0.107895,-0.032249,-0.078025,-0.021458,-0.123813,0.062838,0.061666,-0.012223,-0.067549
4,"[-0.05435901431609755, -0.040716265848554346, ...",Мир,-0.054359,-0.040716,-0.048610,0.067943,0.068528,0.167268,-0.005884,-0.204094,...,-0.091118,-0.190087,-0.004587,0.118081,-0.074080,-0.044480,-0.028896,0.027143,-0.039566,-0.165588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,"[-0.12852551789992317, 0.12083297538361987, -0...",Россия,-0.128526,0.120833,-0.089201,-0.108318,0.215013,0.168679,0.026345,-0.141776,...,-0.129567,-0.032553,0.018072,-0.076139,0.038429,-0.123793,0.041669,0.005085,-0.119853,0.067121
1622,"[-0.08483594141781647, 0.06638847651861687, 0....",Россия,-0.084836,0.066388,0.006405,-0.009657,0.009611,0.169968,-0.084734,-0.096505,...,-0.008029,-0.052033,0.015198,-0.115209,-0.143036,-0.114123,-0.017190,0.002313,0.005189,-0.122192
1623,"[-0.09809835494709122, -0.04048196648960058, -...",Россия,-0.098098,-0.040482,-0.030405,-0.047726,0.076643,0.203821,-0.138404,-0.062946,...,0.052257,0.064651,-0.017520,-0.035399,-0.023216,-0.211095,0.070182,0.098357,-0.038269,-0.127795
1624,"[-0.10410822005594556, 0.10940082387467806, 0....",Мир,-0.104108,0.109401,0.016355,0.057165,0.187137,0.147629,0.005908,-0.235896,...,0.049636,-0.171586,0.021935,-0.053969,-0.074176,-0.094896,-0.008423,0.003542,-0.009126,-0.057312


In [ ]:
result.drop(['text'],axis =1, inplace = True)

In [ ]:
result

,rubric,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300
0,Мир,-0.141926,0.089973,-0.182864,0.091670,0.175368,0.184784,0.028104,-0.156278,-0.197947,...,-0.007197,-0.045465,0.029268,0.089884,-0.115882,-0.180595,-0.045483,-0.030673,0.007729,-0.050785
1,Россия,-0.178737,0.033615,-0.005492,-0.014667,0.077261,0.235475,-0.149112,-0.149998,-0.026083,...,-0.062148,-0.155750,0.076475,-0.038653,-0.199117,-0.201556,-0.074330,0.007207,-0.005383,-0.072535
2,Мир,-0.037623,0.124084,-0.031329,0.059092,0.144818,0.182589,-0.097897,-0.219109,-0.091652,...,-0.075188,-0.130190,0.100777,-0.012934,-0.165758,-0.021185,-0.081217,-0.030957,0.069550,-0.087013
3,Мир,-0.075614,0.032001,-0.051134,0.102946,0.143312,0.287640,-0.108037,-0.233706,-0.146325,...,-0.035773,-0.107895,-0.032249,-0.078025,-0.021458,-0.123813,0.062838,0.061666,-0.012223,-0.067549
4,Мир,-0.054359,-0.040716,-0.048610,0.067943,0.068528,0.167268,-0.005884,-0.204094,0.084872,...,-0.091118,-0.190087,-0.004587,0.118081,-0.074080,-0.044480,-0.028896,0.027143,-0.039566,-0.165588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,Россия,-0.128526,0.120833,-0.089201,-0.108318,0.215013,0.168679,0.026345,-0.141776,-0.174363,...,-0.129567,-0.032553,0.018072,-0.076139,0.038429,-0.123793,0.041669,0.005085,-0.119853,0.067121
1622,Россия,-0.084836,0.066388,0.006405,-0.009657,0.009611,0.169968,-0.084734,-0.096505,-0.118263,...,-0.008029,-0.052033,0.015198,-0.115209,-0.143036,-0.114123,-0.017190,0.002313,0.005189,-0.122192
1623,Россия,-0.098098,-0.040482,-0.030405,-0.047726,0.076643,0.203821,-0.138404,-0.062946,-0.137624,...,0.052257,0.064651,-0.017520,-0.035399,-0.023216,-0.211095,0.070182,0.098357,-0.038269,-0.127795
1624,Мир,-0.104108,0.109401,0.016355,0.057165,0.187137,0.147629,0.005908,-0.235896,-0.052837,...,0.049636,-0.171586,0.021935,-0.053969,-0.074176,-0.094896,-0.008423,0.003542,-0.009126,-0.057312


In [ ]:
result['rubric'].replace({'Мир': 0,'Россия' : 1},inplace = True)



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score





X_train, X_test, y_train, y_test = train_test_split(result.drop("rubric", axis=1), result["rubric"], test_size=0.2, random_state=42)


clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.94


In [ ]:
df = pd.read_csv('/content/news.csv')
df = df[df['rubric'].notna()]
df = df.loc[(df['rubric'] == 'Мир') | (df['rubric'] == 'Россия')]
df = df.drop(['source','title','publication_date','subrubric','tags'],axis =1 )

In [ ]:
def mean_vector2(string):
  len = 0
  vector = np.zeros(300)
  for word in string:
    if type(word) != str:
      continue
    elif word not in model.wv.vocab:
      print(word)
      continue
    vector = vector + model.wv.get_vector(word)
    len+=1


  return vector/len

In [ ]:
import gensim.downloader as api

# Load a pre-trained model (for example, fasttext in Russian)
model = api.load("fasttext-wiki-news-subwords-300")

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
df['text'] = df['text'].map(mean_vector2)
vector_df = pd.DataFrame(df['text'].to_list(), columns=[f'feature_{i+1}' for i in range(300)])

# Concatenate feature DataFrame with rubric column
result  = pd.concat([df.reset_index().drop("index", axis=1), vector_df], axis=1)
result.drop(['text'],axis =1, inplace = True)
result['rubric'].replace({'Мир': 0,'Россия' : 1},inplace = True)

<ipython-input-36-f12daee2bf56>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  elif word not in model.wv.vocab:
<ipython-input-36-f12daee2bf56>:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector = vector + model.wv.get_vector(word)


Выходные данные были обрезаны до нескольких последних строк (5000).
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [ ]:
result

,rubric,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300
0,0,0.075685,0.004382,0.139067,0.028679,-0.048978,0.088437,-0.006073,0.203926,-0.015871,...,0.035146,0.061403,-0.225491,-0.132917,0.052802,-0.031589,0.009965,-0.071924,0.045572,0.082618
1,1,0.077364,0.005725,0.142494,0.025208,-0.044835,0.091299,-0.004981,0.206396,-0.014592,...,0.038354,0.058393,-0.219612,-0.134092,0.052511,-0.033842,0.007797,-0.074051,0.045947,0.087006
2,0,0.079992,0.004701,0.143073,0.028104,-0.045564,0.090876,-0.007949,0.198184,-0.015703,...,0.035953,0.061988,-0.222319,-0.135128,0.054385,-0.034483,0.010809,-0.073933,0.047103,0.086959
3,0,0.080086,0.001177,0.144464,0.035846,-0.049813,0.093705,-0.007734,0.200672,-0.018006,...,0.037215,0.062033,-0.228493,-0.137057,0.057074,-0.035946,0.010792,-0.072208,0.044556,0.085382
4,0,0.079105,0.004315,0.144591,0.032055,-0.046272,0.093622,-0.008312,0.203224,-0.017134,...,0.037277,0.061005,-0.227260,-0.136471,0.054719,-0.035788,0.010975,-0.076733,0.046138,0.089062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,1,0.076240,0.001596,0.142082,0.023175,-0.046439,0.088173,-0.004972,0.199041,-0.016052,...,0.040935,0.059215,-0.213948,-0.134802,0.052328,-0.029442,0.006607,-0.069459,0.043634,0.082685
1622,1,0.078729,0.002549,0.143345,0.032813,-0.041787,0.096082,-0.009845,0.194813,-0.017267,...,0.040908,0.057791,-0.219311,-0.137586,0.055371,-0.035025,0.008323,-0.075881,0.047997,0.087897
1623,1,0.079277,0.002999,0.142095,0.029161,-0.043922,0.091444,-0.006512,0.197385,-0.017127,...,0.040127,0.056075,-0.216624,-0.135448,0.056304,-0.034808,0.006841,-0.073808,0.046541,0.085945
1624,0,0.075249,0.002349,0.140390,0.018603,-0.042940,0.088804,-0.004711,0.199955,-0.015619,...,0.040465,0.057635,-0.211296,-0.134257,0.052904,-0.028332,0.007894,-0.069327,0.044871,0.083315


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(result.drop("rubric", axis=1), result["rubric"], test_size=0.2, random_state=42)


clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.64


#Для "своей" модели брал вектор размерности 300, так как fasttext-wiki-news-subwords-300 имеет размерность 300. Можно увидеть, что на данном датасете победила моя модель. Однако предобученная модель очень хорошо справилась для незнакомого текста


##Задание 4


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.4 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
df = pd.read_csv('/content/IMDB data.csv')

In [ ]:
df['sentiment'].replace({'negative': 0,'positive' : 1},inplace = True)

In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
text = df.review.values
labels = df.sentiment.values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
token_id[6]

tensor([  101,  1045,  2469,  2052,  2066,  2000,  2156,  1037, 15218,  1997,
         1037,  2039,  6052,  2712, 17157,  2102,  2186,  2007,  1996,  6627,
         2027,  2031,  2651,  2009,  2052,  3288,  2067,  1996,  4845,  8277,
         1999,   102])

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒═════════════╤═════════════╤══════════════════╕
│ Tokens      │   Token IDs │   Attention Mask │
╞═════════════╪═════════════╪══════════════════╡
│ [CLS]       │         101 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ if          │        2065 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ sicily      │       12071 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ is          │        2003 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ a           │        1037 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ territory   │        3700 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ of          │        1997 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ the         │        1996 │                1 │
├─────────────┼─────────────┼──────────────────┤
│ baroque     │       10456 │                1 │
├─────────────┼─────

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#Первая модель


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  50%|█████     | 1/2 [05:37<05:37, 337.84s/it]


	 - Train loss: 0.4945
	 - Validation Accuracy: 0.7816
	 - Validation Precision: 0.7942
	 - Validation Recall: 0.7598
	 - Validation Specificity: 0.7969



Epoch: 100%|██████████| 2/2 [11:10<00:00, 335.26s/it]


	 - Train loss: 0.3511
	 - Validation Accuracy: 0.7833
	 - Validation Precision: 0.7347
	 - Validation Recall: 0.8805
	 - Validation Specificity: 0.6807



#Можем заметить что у модели очень хороший recall

In [ ]:
def predict(new_sentence):
  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []

  # Apply the tokenizer
  encoding = preprocessing(new_sentence, tokenizer)

  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

  prediction = 'Nice film bro ' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Bad film Bro'

  print('Input Sentence: ', new_sentence)
  print('Predicted Class: ', prediction)

In [ ]:
new_sentence = 'Amazing film, i like it, breathtaking'
predict(new_sentence)
new_sentence = "So boring, i cannot see that"
predict(new_sentence)
new_sentence = "Bruh, i am upset"
predict(new_sentence)

Input Sentence:  Amazing film, i like it, breathtaking
Predicted Class:  Nice film bro 
Input Sentence:  So boring, i cannot see that
Predicted Class:  Bad film Bro
Input Sentence:  Bruh, i am upset
Predicted Class:  Bad film Bro


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Вторая модель. увеличил lr и количество эпох

In [ ]:
model2 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
    
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model2.parameters(), 
                              lr = 3e-5,
                              eps = 1e-08
                              )

# Run on GPU


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model2.to(device)
print(device)
# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 3

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model2 to training mode
    model2.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model2(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model2 to evaluation mode
    model2.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model2(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

cuda:0


Epoch:  33%|███▎      | 1/3 [05:34<11:08, 334.01s/it]


	 - Train loss: 0.4817
	 - Validation Accuracy: 0.7867
	 - Validation Precision: 0.7542
	 - Validation Recall: 0.8514
	 - Validation Specificity: 0.7175



Epoch:  67%|██████▋   | 2/3 [11:08<05:34, 334.12s/it]


	 - Train loss: 0.3300
	 - Validation Accuracy: 0.7952
	 - Validation Precision: 0.7628
	 - Validation Recall: 0.8560
	 - Validation Specificity: 0.7305



Epoch: 100%|██████████| 3/3 [16:40<00:00, 333.63s/it]


	 - Train loss: 0.1794
	 - Validation Accuracy: 0.7928
	 - Validation Precision: 0.8051
	 - Validation Recall: 0.7714
	 - Validation Specificity: 0.8079



In [ ]:
def predict2(new_sentence):
  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []

  # Apply the tokenizer
  encoding = preprocessing(new_sentence, tokenizer)

  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model2(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

  prediction = 'Nice film bro ' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Bad film Bro'

  print('Input Sentence: ', new_sentence)
  print('Predicted Class: ', prediction)

In [ ]:
new_sentence = 'I recently watched the latest blockbuster and I must say, it did not disappoint. The acting was superb and the special effects were truly impressive. The plot kept me on the edge of my seat and I found myself fully invested in the characters journeys. Overall, a great film that I would highly recommend.'
predict2(new_sentence)
new_sentence = "I had high expectations for this movie, but unfortunately it fell short. The story was lackluster and the acting was mediocre at best. The special effects were impressive, but that alone couldn't make up for the lack of depth in the plot. Overall, I was disappointed and wouldn't recommend this film"
predict2(new_sentence)
new_sentence = "This movie was a pleasant surprise! I didn't have high expectations going in, but the story was engaging and the characters were well-developed. The acting was top-notch and the special effects were impressive. I found myself fully invested in the story and cared about the characters' fates. Overall, a great film that I would definitely recommend"
predict2(new_sentence)

Input Sentence:  I recently watched the latest blockbuster and I must say, it did not disappoint. The acting was superb and the special effects were truly impressive. The plot kept me on the edge of my seat and I found myself fully invested in the characters journeys. Overall, a great film that I would highly recommend.
Predicted Class:  Nice film bro 
Input Sentence:  I had high expectations for this movie, but unfortunately it fell short. The story was lackluster and the acting was mediocre at best. The special effects were impressive, but that alone couldn't make up for the lack of depth in the plot. Overall, I was disappointed and wouldn't recommend this film
Predicted Class:  Bad film Bro
Input Sentence:  This movie was a pleasant surprise! I didn't have high expectations going in, but the story was engaging and the characters were well-developed. The acting was top-notch and the special effects were impressive. I found myself fully invested in the story and cared about the charac

# Третья модель

In [ ]:
model3 = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
    
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model3.parameters(), 
                              lr = 1e-5,
                              eps = 1e-08
                              )

# Run on GPU


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model3.to(device)
print(device)
# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 5

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model3 to training mode
    model3.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model3(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model2 to evaluation mode
    model3.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model2(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

cuda:0


Epoch:  20%|██        | 1/5 [05:32<22:08, 332.25s/it]


	 - Train loss: 0.4837
	 - Validation Accuracy: 0.7928
	 - Validation Precision: 0.8051
	 - Validation Recall: 0.7714
	 - Validation Specificity: 0.8079



Epoch:  40%|████      | 2/5 [11:04<16:37, 332.35s/it]


	 - Train loss: 0.3640
	 - Validation Accuracy: 0.7928
	 - Validation Precision: 0.8051
	 - Validation Recall: 0.7714
	 - Validation Specificity: 0.8079



...


	 - Train loss: 0.2473
	 - Validation Accuracy: 0.7928
	 - Validation Precision: 0.8051
	 - Validation Recall: 0.7714
	 - Validation Specificity: 0.8079



Epoch:  80%|████████  | 4/5 [22:10<05:32, 332.61s/it]


	 - Train loss: 0.1419
	 - Validation Accuracy: 0.7928
	 - Validation Precision: 0.8051
	 - Validation Recall: 0.7714
	 - Validation Specificity: 0.8079



Epoch: 100%|██████████| 5/5 [27:42<00:00, 332.58s/it]


	 - Train loss: 0.0840
	 - Validation Accuracy: 0.7928
	 - Validation Precision: 0.8051
	 - Validation Recall: 0.7714
	 - Validation Specificity: 0.8079



# У этой модели почему-то не менялись параметры кроме train loss, не смог понять почему так, но вероятно она переобучилась

In [ ]:
def predict3(new_sentence):
  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []

  # Apply the tokenizer
  encoding = preprocessing(new_sentence, tokenizer)

  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model3(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

  prediction = 'Nice film bro ' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Bad film Bro'

  print('Input Sentence: ', new_sentence)
  print('Predicted Class: ', prediction)

In [ ]:
new_sentence = 'I recently watched the latest blockbuster and I must say, it did not disappoint. The acting was superb and the special effects were truly impressive. The plot kept me on the edge of my seat and I found myself fully invested in the characters journeys. Overall, a great film that I would highly recommend.'
predict3(new_sentence)
new_sentence = "I had high expectations for this movie, but unfortunately it fell short. The story was lackluster and the acting was mediocre at best. The special effects were impressive, but that alone couldn't make up for the lack of depth in the plot. Overall, I was disappointed and wouldn't recommend this film"
predict3(new_sentence)
new_sentence = "This movie was a pleasant surprise! I didn't have high expectations going in, but the story was engaging and the characters were well-developed. The acting was top-notch and the special effects were impressive. I found myself fully invested in the story and cared about the characters' fates. Overall, a great film that I would definitely recommend"
predict3(new_sentence)

Input Sentence:  I recently watched the latest blockbuster and I must say, it did not disappoint. The acting was superb and the special effects were truly impressive. The plot kept me on the edge of my seat and I found myself fully invested in the characters journeys. Overall, a great film that I would highly recommend.
Predicted Class:  Nice film bro 
Input Sentence:  I had high expectations for this movie, but unfortunately it fell short. The story was lackluster and the acting was mediocre at best. The special effects were impressive, but that alone couldn't make up for the lack of depth in the plot. Overall, I was disappointed and wouldn't recommend this film
Predicted Class:  Bad film Bro
Input Sentence:  This movie was a pleasant surprise! I didn't have high expectations going in, but the story was engaging and the characters were well-developed. The acting was top-notch and the special effects were impressive. I found myself fully invested in the story and cared about the charac

# Вывод : Настройка параметров Bert, довольно сложная и вычислительно долгая процедура. Все модели получились +- одинаковые по качеству